# Preprocessing Consumer Data

This Notebook Preprocesses The Consumer Data 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DateType
from datetime import datetime, timedelta

In [2]:
spark = (
    SparkSession.builder.appName('Consumer Data Preprocess')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/13 21:52:57 WARN Utils: Your hostname, DESKTOP-F216TKE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/13 21:52:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/13 21:52:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/13 21:52:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/13 21:52:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
tbl_consumer =  spark.read.option("header", True).option("delimiter", "|").csv(".././data/tables/tbl_consumer.csv")

In [7]:
tbl_consumer.show()

+-----------------+--------------------+-----+--------+-----------+-----------+
|             name|             address|state|postcode|     gender|consumer_id|
+-----------------+--------------------+-----+--------+-----------+-----------+
| Yolanda Williams|413 Haney Gardens...|   WA|    6935|     Female|    1195503|
|       Mary Smith|     3764 Amber Oval|  NSW|    2782|     Female|     179208|
|    Jill Jones MD|  40693 Henry Greens|   NT|     862|     Female|    1194530|
|  Lindsay Jimenez|00653 Davenport C...|  NSW|    2780|     Female|     154128|
|Rebecca Blanchard|9271 Michael Mano...|   WA|    6355|     Female|     712975|
|    Karen Chapman|2706 Stewart Oval...|  NSW|    2033|     Female|     407340|
|     Andrea Jones|   122 Brandon Cliff|  QLD|    4606|     Female|     511685|
| Stephen Williams|6804 Wright Crest...|   WA|    6056|       Male|     448088|
|  Stephanie Reyes|5813 Denise Land ...|  NSW|    2482|     Female|     650435|
| Jillian Gonzales|461 Ryan Common S...|

Since Address is not valid it has been dropped

In [8]:
tbl_consumer = tbl_consumer.drop("address")

In [9]:
tbl_consumer.count()

499999

In [10]:
print(tbl_consumer.select("state").distinct())
print(tbl_consumer.select("gender").distinct())

+-----+
|state|
+-----+
|   NT|
|  ACT|
|   SA|
|  TAS|
|   WA|
|  QLD|
|  VIC|
|  NSW|
+-----+

+-----------+
|     gender|
+-----------+
|Undisclosed|
|     Female|
|       Male|
+-----------+



In [11]:
tbl_consumer = tbl_consumer.filter(F.col("state").isin(["NSW", "WA", "NT", "VIC", "TAS", "QLD", "SA",   "ACT"]))

In [12]:
tbl_consumer = tbl_consumer.filter(F.col("gender").isin(["Female", "Male", "Undisclosed"]))

In [13]:
tbl_consumer.printSchema()

root
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: string (nullable = true)



Check if all postcodes are valid

In [14]:
tbl_consumer.filter(
    ((F.col("postcode") >= 1000) & (F.col("postcode") <= 1999)) |  
    ((F.col("postcode") >= 2000) & (F.col("postcode") <= 2599)) |
    ((F.col("postcode") >= 2619) & (F.col("postcode") <= 2899)) |
    ((F.col("postcode") >= 2921) & (F.col("postcode") <= 2999)) |
    ((F.col("postcode") >= 200) & (F.col("postcode") <= 299)) |  
    ((F.col("postcode") >= 2600) & (F.col("postcode") <= 2618)) |
    ((F.col("postcode") >= 2900) & (F.col("postcode") <= 2920)) |
    ((F.col("postcode") >= 3000) & (F.col("postcode") <= 3999)) |
    ((F.col("postcode") >= 8000) & (F.col("postcode") <= 8999)) |  
    ((F.col("postcode") >= 4000) & (F.col("postcode") <= 4999)) |
    ((F.col("postcode") >= 9000) & (F.col("postcode") <= 9999)) |  
    ((F.col("postcode") >= 5000) & (F.col("postcode") <= 5799)) |
    ((F.col("postcode") >= 5800) & (F.col("postcode") <= 5999)) |  
    ((F.col("postcode") >= 6000) & (F.col("postcode") <= 6797)) |
    ((F.col("postcode") >= 6800) & (F.col("postcode") <= 6999)) |  
    ((F.col("postcode") >= 7000) & (F.col("postcode") <= 7799)) |
    ((F.col("postcode") >= 7800) & (F.col("postcode") <= 7999)) |  
    ((F.col("postcode") >= 800) & (F.col("postcode") <= 899)) |
    ((F.col("postcode") >= 900) & (F.col("postcode") <= 999))
).count()

499690

In [15]:
sdf = tbl_consumer.filter(
    ((F.col("postcode") >= 1000) & (F.col("postcode") <= 1999)) |  
    ((F.col("postcode") >= 2000) & (F.col("postcode") <= 2599)) |
    ((F.col("postcode") >= 2619) & (F.col("postcode") <= 2899)) |
    ((F.col("postcode") >= 2921) & (F.col("postcode") <= 2999)) |
    ((F.col("postcode") >= 200) & (F.col("postcode") <= 299)) |  
    ((F.col("postcode") >= 2600) & (F.col("postcode") <= 2618)) |
    ((F.col("postcode") >= 2900) & (F.col("postcode") <= 2920)) |
    ((F.col("postcode") >= 3000) & (F.col("postcode") <= 3999)) |
    ((F.col("postcode") >= 8000) & (F.col("postcode") <= 8999)) |  
    ((F.col("postcode") >= 4000) & (F.col("postcode") <= 4999)) |
    ((F.col("postcode") >= 9000) & (F.col("postcode") <= 9999)) |  
    ((F.col("postcode") >= 5000) & (F.col("postcode") <= 5799)) |
    ((F.col("postcode") >= 5800) & (F.col("postcode") <= 5999)) |  
    ((F.col("postcode") >= 6000) & (F.col("postcode") <= 6797)) |
    ((F.col("postcode") >= 6800) & (F.col("postcode") <= 6999)) |  
    ((F.col("postcode") >= 7000) & (F.col("postcode") <= 7799)) |
    ((F.col("postcode") >= 7800) & (F.col("postcode") <= 7999)) |  
    ((F.col("postcode") >= 800) & (F.col("postcode") <= 899)) |
    ((F.col("postcode") >= 900) & (F.col("postcode") <= 999))
)

In [16]:
sdf.count()

499690

In [17]:
tbl_consumer.subtract(sdf)

name,state,postcode,gender,consumer_id
Holly Patel,WA,6798,Female,526854
Michelle Morrison,WA,6799,Female,680827
Sarah Underwood,WA,6798,Female,906266
Clifford Roberts,WA,6799,Male,1191458
Heidi Brown,WA,6798,Female,76750
Jorge Bennett,WA,6798,Male,81388
Samuel Porter,WA,6799,Undisclosed,655248
Jason Lane,WA,6799,Male,911214
David Keller,WA,6798,Male,1267439
Jennifer Lopez,WA,6798,Female,546210


All postcodes are considered valid ; even those for sparesely populated islands 

In [18]:
tbl_consumer = tbl_consumer.withColumn("postcode", tbl_consumer["postcode"].cast(IntegerType()))

In [19]:
print("Number of unique consumer_id: "+str(tbl_consumer.select("consumer_id").distinct().count()))
print("Number of counts after preprocessing: " + str(tbl_consumer.count()))
tbl_consumer = tbl_consumer.withColumnRenamed("name","consumer_name")

Number of unique consumer_id: 499999
Number of counts after preprocessing: 499999


In [20]:
tbl_consumer.show()

+-----------------+-----+--------+-----------+-----------+
|    consumer_name|state|postcode|     gender|consumer_id|
+-----------------+-----+--------+-----------+-----------+
| Yolanda Williams|   WA|    6935|     Female|    1195503|
|       Mary Smith|  NSW|    2782|     Female|     179208|
|    Jill Jones MD|   NT|     862|     Female|    1194530|
|  Lindsay Jimenez|  NSW|    2780|     Female|     154128|
|Rebecca Blanchard|   WA|    6355|     Female|     712975|
|    Karen Chapman|  NSW|    2033|     Female|     407340|
|     Andrea Jones|  QLD|    4606|     Female|     511685|
| Stephen Williams|   WA|    6056|       Male|     448088|
|  Stephanie Reyes|  NSW|    2482|     Female|     650435|
| Jillian Gonzales|  VIC|    3220|     Female|    1058499|
|     Eugene Lucas|  VIC|    3063|Undisclosed|     428325|
|    Melissa Jones|   WA|    6743|     Female|    1494640|
| Angela Brown PhD|  QLD|    4673|     Female|    1146717|
|     Lance Butler|  VIC|    3332|       Male|    134354

In [21]:
tbl_consumer.filter(F.isnull('consumer_name')).count()

0

In [22]:
tbl_consumer.filter(F.isnull('gender')).count()

0

In [23]:
tbl_consumer.filter(F.isnull('postcode')).count()

0

In [24]:
tbl_consumer.filter(F.isnull('consumer_id')).count()

0

In [25]:
tbl_consumer.filter(F.isnull('state')).count()

0

Consumer Details Dataframe Preprocess

In [26]:
consumer_details = spark.read.parquet(".././data/tables/consumer_user_details.parquet")
consumer_details.show()

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|      1|    1195503|
|      2|     179208|
|      3|    1194530|
|      4|     154128|
|      5|     712975|
|      6|     407340|
|      7|     511685|
|      8|     448088|
|      9|     650435|
|     10|    1058499|
|     11|     428325|
|     12|    1494640|
|     13|    1146717|
|     14|    1343547|
|     15|    1463076|
|     16|    1356405|
|     17|    1331093|
|     18|      80965|
|     19|    1226530|
|     20|    1390367|
+-------+-----------+
only showing top 20 rows



In [27]:
consumer_details.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)



Change user_id and consumer_id to string type 

In [28]:
consumer_details = consumer_details.withColumn('user_id', F.col('user_id').cast('string'))
consumer_details = consumer_details.withColumn('consumer_id', F.col('consumer_id').cast('string'))

In [29]:
consumer_details.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- consumer_id: string (nullable = true)



In [30]:
consumer_details.show(1, vertical=True, truncate=False)

-RECORD 0--------------
 user_id     | 1       
 consumer_id | 1195503 
only showing top 1 row



Look for Dupilcate Values

In [31]:
consumer_details.count()

499999

In [32]:
#consumer_details.dropDuplicates().count()
consumer_details.select('user_id').distinct().count()

499999

In [33]:
consumer_details.select('consumer_id').distinct().count()

499999

No duplicate values have been found

Check for Missing Values 

In [34]:
consumer_details.filter(F.isnull('consumer_id')).count()


0

In [35]:
consumer_details.filter(F.isnan('consumer_id')).count()

0

In [36]:
consumer_details.filter(F.isnull('user_id')).count()

0

In [37]:
consumer_details.filter(F.isnan('user_id')).count()

0

In [38]:
consumer_details.write.mode('overwrite').parquet('.././data/curated/consumer_details')

In [39]:
consumer_fraud = spark.read.option("header", True).csv(".././data/tables/consumer_fraud_probability.csv")
consumer_fraud.show()

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
|  16556|    2022-02-20|89.65663294494827|
|  10278|    2021-09-28|83.59136689427714|
|  15790|    2021-12-30|71.77065889280253|
|   5233|    2021-08-29|85.87123303878818|
|    230|    2021-08-28|86.28328808934151|
|  13601|    2021-12-26|83.13696487489679|
|   6383|    2021-09-15| 66.2676451623754|
|   3513|    2022-02-27|75.16981192247916|
|  18658|    2021-10-19|82.98609082999361|
|   5965|    2021-11-14|69.37164467869053|
|  18714|    2021-11-14|83.78813794627237|
|  22957|    2022-02-12|82.79065699075498|
|  20118|    2021-09-05|80.34030486265003|
|   6436|    2021-12-24|84.81618344606828|
|  17900|    2022-02-25|92.73262811161372|
+-------+--

In [40]:
consumer_fraud.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- fraud_probability: string (nullable = true)



In [41]:
consumer_fraud = consumer_fraud.withColumn('order_datetime', F.col('order_datetime').cast(DateType()))

In [42]:
consumer_fraud = consumer_fraud.withColumn('fraud_probability', F.col('fraud_probability').cast('double'))

In [43]:
consumer_fraud.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- fraud_probability: double (nullable = true)



Check the range for fraude probability

In [44]:
consumer_fraud.filter(F.col('fraud_probability') < 0).count()

0

In [45]:
consumer_fraud.filter(F.col('fraud_probability') > 100).count()

0

The ranges of fraud probability lies in the correct range

Ensure all user_id's are properly referenced 

In [46]:
userids = consumer_fraud.select(F.col('user_id')).distinct()

In [47]:
userids = userids.toPandas()

In [48]:

userids = userids['user_id'].tolist()

In [49]:
userids

['7252',
 '1512',
 '14157',
 '2294',
 '9586',
 '21248',
 '17401',
 '16250',
 '2904',
 '22596',
 '9009',
 '21783',
 '18574',
 '18306',
 '15634',
 '18556',
 '21452',
 '19095',
 '22254',
 '14887',
 '22121',
 '1159',
 '6240',
 '20868',
 '6194',
 '22728',
 '21331',
 '18726',
 '21889',
 '16320',
 '3606',
 '829',
 '14838',
 '18509',
 '21249',
 '7762',
 '9583',
 '2069',
 '20158',
 '14204',
 '2088',
 '15271',
 '17686',
 '8304',
 '21259',
 '18947',
 '5645',
 '21894',
 '17427',
 '4937',
 '296',
 '14899',
 '14369',
 '12529',
 '23918',
 '3210',
 '675',
 '13192',
 '17714',
 '23843',
 '5325',
 '10436',
 '1572',
 '23097',
 '15555',
 '4821',
 '20512',
 '4032',
 '9993',
 '2162',
 '19338',
 '467',
 '3959',
 '12847',
 '11563',
 '11888',
 '10096',
 '18992',
 '11078',
 '15269',
 '1090',
 '7273',
 '6731',
 '6613',
 '11722',
 '9030',
 '23459',
 '691',
 '10351',
 '2136',
 '18634',
 '13282',
 '20219',
 '23318',
 '20428',
 '5925',
 '853',
 '14879',
 '23479',
 '5067',
 '19289',
 '3015',
 '6558',
 '16549',
 '11313

In [50]:
detailsuserid = consumer_details.select(F.col('user_id')).distinct()
detailsuserid = detailsuserid.toPandas()
detailsuserid = detailsuserid['user_id'].tolist()

In [51]:
print(len(set(userids).difference(detailsuserid)))

0


In [52]:
consumersusers = tbl_consumer.select(F.col('consumer_id')).distinct()
consumersusers = consumersusers.toPandas()
consumersusers = consumersusers['consumer_id'].tolist()

In [53]:
detailsconid = consumer_details.select(F.col('consumer_id')).distinct()
detailsconid = detailsconid.toPandas()
detailsconid = detailsconid['consumer_id'].tolist()

In [54]:
print(len(set(detailsconid).difference(consumersusers)))

0


In [55]:
consumer_details.write.mode('overwrite').parquet('.././data/curated/consumer_details')

In [56]:
tbl_consumer.write.mode('overwrite').parquet('.././data/curated/tbl_consumer')

In [57]:
tbl_consumer.filter(F.col('consumer_id') == '1058206').show()

+--------------+-----+--------+------+-----------+
| consumer_name|state|postcode|gender|consumer_id|
+--------------+-----+--------+------+-----------+
|Anthony Powell|   WA|    6798|  Male|    1058206|
+--------------+-----+--------+------+-----------+



In [58]:
consumer_details.show()

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|      1|    1195503|
|      2|     179208|
|      3|    1194530|
|      4|     154128|
|      5|     712975|
|      6|     407340|
|      7|     511685|
|      8|     448088|
|      9|     650435|
|     10|    1058499|
|     11|     428325|
|     12|    1494640|
|     13|    1146717|
|     14|    1343547|
|     15|    1463076|
|     16|    1356405|
|     17|    1331093|
|     18|      80965|
|     19|    1226530|
|     20|    1390367|
+-------+-----------+
only showing top 20 rows



In [59]:
consumer_details.filter(F.col('consumer_id') == '546210').show()

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|   1681|     546210|
+-------+-----------+



There are no additional user_id's that have not been referenced 

In [60]:
consumer_fraud.write.mode('overwrite').parquet('.././data/curated/consumer_fraud')